# 이상치 제거 및 데이터 정제

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
DATA_PATH = '/content/drive/MyDrive/멋사/mid project/'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
org_df = pd.read_csv(DATA_PATH + 'train.csv')
org_df

### 좌석수 조인

In [ ]:
hall_df = pd.read_csv(DATA_PATH + '공연시설별좌석수.csv')
hall_df.head()

In [ ]:
# 원본에 공연장 정보 조인
org_hall_df = pd.merge(org_df, hall_df, on='PBLPRFR_PLACE_NM', how='left')
org_hall_df.head()

### URL 정보 삭제

In [ ]:
org_hall_df.drop('HMPG_URL', axis=1, inplace=True)
org_hall_df.head()

- 나이값은 '대' 를 빼고 나이값만 int로
- 성별은 M -> 0, F -> 1
- PBLPRFR_DE는 date값으로

In [ ]:
org_hall_df['BN_VALUE'] = org_hall_df['BN_VALUE'].str.extract('(\d+)').astype(float).fillna(0).astype(int)
org_hall_df['SEXDSTN_VALUE'] = org_hall_df['SEXDSTN_VALUE'].replace({'M': 0, 'F': 1})
org_hall_df['PBLPRFR_DE'] = pd.to_datetime(org_hall_df['PBLPRFR_DE'], format='%Y%m%d', errors='coerce')
org_hall_df.head()

In [ ]:
# prompt: org_hall_df[SEXDSTN_VALUE]값들을 int로 변환

# SEXDSTN_VALUE 컬럼의 값을 int로 변환 (숫자로 변환 가능한 값만 변환)
org_hall_df['SEXDSTN_VALUE'] = pd.to_numeric(org_hall_df['SEXDSTN_VALUE'], errors='coerce').fillna(1).astype(int)

In [ ]:
org_hall_df.info()

In [ ]:
org_hall_df.head()

### 크롤링 데이터 가져오기

In [ ]:
delete_df = pd.read_csv(DATA_PATH + 'crolling_result.csv')
delete_df

- True, None값 골라내기

In [ ]:
mask = (delete_df['result'] == False)
delete_df = delete_df[~mask]
delete_df.head()

In [ ]:
len(delete_df), len(org_hall_df) # 원래는 142, 3664663

In [ ]:
d_org_df = org_hall_df.copy()
d_org_df

- 환불여부 0, 1로 변환

In [ ]:
# RFN에서 N = 0, Y = 1로 변환
d_org_df.RFN_AT = d_org_df.RFN_AT.replace({'N': 0, 'Y': 1})
d_org_df.head()

In [ ]:
#masking = d_org_df.merge(delete_df[['PBLPRFR_NM', 'PBLPRFR_DE']], on=['PBLPRFR_NM', 'PBLPRFR_DE'], how='inner')
delete_df['PBLPRFR_DE'] = pd.to_datetime(delete_df['PBLPRFR_DE'])

### 'PBLPRFR_NM', 'PBLPRFR_DE' 가 같은 행들 마스킹

In [ ]:
masking = d_org_df.merge(delete_df[['PBLPRFR_NM', 'PBLPRFR_DE']], on=['PBLPRFR_NM', 'PBLPRFR_DE'], how='inner')

In [ ]:
print(len(masking))
masking

In [ ]:
d_org_df = d_org_df[~d_org_df[['PBLPRFR_NM', 'PBLPRFR_DE']].apply(tuple, axis=1).isin(masking[['PBLPRFR_NM', 'PBLPRFR_DE']].apply(tuple, axis=1))]
d_org_df

### 삭제완료

In [ ]:
len(d_org_df) == len(org_hall_df) - len(masking)

# 컬럼명 수정
### d_org_df로 수정함

- PBLPRFR_NM : 공연이름
- PBLPRFR_PLACE_NM : 공연장소
- PBLPRFR_DE : 공연날짜
- PBLPRFR_TIME : 공연시간
- GENRE_NM : 장르
- PRC_CN : 가격대
- SETLE_DE : 구매날짜
- SETLE_TIME : 구매시간
- DSCNT_CN : 할인정보
- SETLE_PRICE : 구매가격
- MBER_GRAD_NM : 회원등급
- BN_VALUE : 나이
- SEXDSTN_VALUE : 성별
- OC_AREA_NM : 거주지
- RFN_AT : 환불여부

In [ ]:
d_org_df = d_org_df.rename(columns={'PBLPRFR_NM': '공연이름'})
d_org_df = d_org_df.rename(columns={'PBLPRFR_PLACE_NM': '공연장소'})
d_org_df = d_org_df.rename(columns={'PBLPRFR_DE': '공연날짜'})
d_org_df = d_org_df.rename(columns={'PBLPRFR_TIME': '공연시간'})
d_org_df = d_org_df.rename(columns={'GENRE_NM': '장르'})
d_org_df = d_org_df.rename(columns={'PRC_CN': '가격대'})
d_org_df = d_org_df.rename(columns={'SETLE_DE': '구매날짜'})
d_org_df = d_org_df.rename(columns={'SETLE_TIME': '구매시간'})
d_org_df = d_org_df.rename(columns={'DSCNT_CN': '할인정보'})
d_org_df = d_org_df.rename(columns={'SETLE_PRICE': '구매가격'})
d_org_df = d_org_df.rename(columns={'MBER_GRAD_NM': '회원등급'})
d_org_df = d_org_df.rename(columns={'BN_VALUE': '나이'})
d_org_df = d_org_df.rename(columns={'SEXDSTN_VALUE': '성별'})
d_org_df = d_org_df.rename(columns={'OC_AREA_NM': '거주지'})
d_org_df = d_org_df.rename(columns={'RFN_AT': '환불여부'})
d_org_df.head()

- 우선 csv로 저장후 다른코랩에서 진행
- 램이 너무 무거워짐


In [ ]:
# file_save = DATA_PATH + 'first_refine.csv'
# d_org_df.to_csv(file_save, index=False)

In [ ]:
# prompt: 공연이름에 '공연취소'라고 적혀있는 공연 검색

# 공연이름에 '공연취소'가 포함된 행만 추출
canceled_performances = d_org_df[d_org_df['공연이름'].str.contains('공연취소', na=False)]

# 결과 출력 (예: 처음 5개 행 출력)
print(canceled_performances.head())